<a href="https://colab.research.google.com/github/pi-mishra/PNB_stock_prediction/blob/main/PNB_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('/content/drive/MyDrive/yes_bank/PNB.NS.csv')

In [19]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-01,9.60,9.74,9.55,9.57,6.875042,825970.0
1,2002-07-02,9.60,9.67,9.45,9.50,6.824753,653405.0
2,2002-07-03,9.46,9.57,9.46,9.51,6.831936,305880.0
3,2002-07-04,9.56,9.90,9.52,9.72,6.982801,2413155.0
4,2002-07-05,9.90,9.95,9.60,9.85,7.076193,3460955.0


In [20]:
df.shape

(5215, 7)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5215 entries, 0 to 5214
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5215 non-null   object 
 1   Open       5191 non-null   float64
 2   High       5191 non-null   float64
 3   Low        5191 non-null   float64
 4   Close      5191 non-null   float64
 5   Adj Close  5191 non-null   float64
 6   Volume     5191 non-null   float64
dtypes: float64(6), object(1)
memory usage: 285.3+ KB


In [22]:
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d').dt.strftime('%d-%m-%Y')